In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from statsmodels.stats.diagnostic import het_breuschpagan

pitch = pd.read_excel("corrected_pitch.xlsx")
gk = pd.read_excel("corrected_gk.xlsx")

In [12]:
def list_reduced(lista, mains):
    new_list = []
    for c in lista:
        if c in mains:
            new_list.append(c)
        else:
            new_list.append("Otros")
    return new_list

mains = ["Inglaterra", "Francia", "España", "Italia", "Alemania", "Portugal", "Países Bajos", "Argentina", "Brasil"]

mains_team = ["Inter", "AC Milan", "Juventus", "Napoli", "Paris SG", "Real Madrid", "Barcelona", "Atlético Madrid",
             "Bayern Munich", "Bor. Dortmund", "Chelsea", "Arsenal", "Man City", "Man Utd", "Liverpool"]

c_pitch_from_reduced = list_reduced(pitch.Country_from, mains)
c_pitch_to_reduced = list_reduced(pitch.Country_to, mains)

pitch["Reduced_country_from"] = c_pitch_from_reduced
pitch["Reduced_country_to"] = c_pitch_to_reduced

t_pitch_from_reduced = list_reduced(pitch.Team_from, mains_team)
t_pitch_to_reduced = list_reduced(pitch.Team_to, mains_team)

pitch["Reduced_team_from"] = t_pitch_from_reduced
pitch["Reduced_team_to"] = t_pitch_to_reduced

c_gk_from_reduced = list_reduced(gk.Country_from, mains)
c_gk_to_reduced = list_reduced(gk.Country_to, mains)

gk["Reduced_country_from"] = c_gk_from_reduced
gk["Reduced_country_to"] = c_gk_to_reduced

t_gk_from_reduced = list_reduced(gk.Team_from, mains_team)
t_gk_to_reduced = list_reduced(gk.Team_to, mains_team)

gk["Reduced_team_from"] = t_gk_from_reduced
gk["Reduced_team_to"] = t_gk_to_reduced

gk["Save_perc"] = gk.apply(lambda row: round((row['SoTA'] - row['GA'])/row['SoTA'], 2) if row['SoTA'] > 0 else 0, axis=1)
gk["Save_perc_BC"] = gk.apply(lambda row: round((row['SoTA_BC'] - row['GA_BC'])/row['SoTA_BC'], 2) if row['SoTA_BC'] > 0 else 0, axis=1)
gk["Save_perc_LS"] = gk.apply(lambda row: round((row['SoTA_LS'] - row['GA_LS'])/row['SoTA_LS'], 2) if row['SoTA_LS'] > 0 else 0, axis=1)

In [13]:
categoric_pitch = ['Reduced_team_from', 'Reduced_team_to', 'Reduced_country_from', 'Reduced_country_to', 'Position']

pitch_vars = ['Transfer_value', 'Exp_contr',
       'Age', 'Minutes_pl', 'Minutes_pl_BC', 'NP_goals',
       'Pen_goals', 'Pen_goals_BC', 'Assists', 'Interceptions', 'GCA_BC',
       'Yellow_cards', 'Red_cards']

categoric_gk = ['Team_from', 'Team_to', 'Country_from', 'Country_to']
                   
gk_vars = ['Transfer_value', 'Exp_contr', 'Age', 'Minutes_pl', 'Minutes_pl_LS', 'Minutes_pl_BC',
       'Save_perc', 'Save_perc_BC', 'Save_perc_LS', 'PKA', 'PKSv',
       'Assists', 'Yellow_cards', 'Red_cards']

In [14]:
mains = ["Otros", "Inglaterra", "Francia", "España", "Italia", "Alemania", "Portugal", "Países Bajos", "Argentina", "Brasil"]

mains_team = ["Otros", "Inter", "AC Milan", "Juventus", "Napoli", "Paris SG", "Real Madrid", "Barcelona", "Atlético Madrid",
             "Bayern Munich", "Bor. Dortmund", "Chelsea", "Arsenal", "Man City", "Man Utd", "Liverpool"]


pos = ['Right-Back', 'Centre-Forward', 'Left-Back', 'Central Midfield', 
       'Centre-Back', 'Right Winger', 'Attacking Midfield',
       'Defensive Midfield', 'Left Winger', 'Second Striker', 'Left Midfield', 'Right Midfield']

pitch["Reduced_team_from"] = pd.Categorical(pitch["Reduced_team_from"], categories=mains_team)
df_dummies = pd.get_dummies(pitch[pitch_vars + categoric_pitch], columns=["Reduced_team_from"], drop_first=True)

df_dummies["Reduced_team_to"] = pd.Categorical(pitch["Reduced_team_to"], categories=mains_team)
df_dummies = pd.get_dummies(df_dummies, columns=["Reduced_team_to"], drop_first=True)

df_dummies["Reduced_country_from"] = pd.Categorical(pitch["Reduced_country_from"], categories=mains)
df_dummies = pd.get_dummies(df_dummies, columns=["Reduced_country_from"], drop_first=True)

df_dummies["Reduced_country_to"] = pd.Categorical(pitch["Reduced_country_to"], categories=mains)
df_dummies = pd.get_dummies(df_dummies, columns=["Reduced_country_to"], drop_first=True)

df_dummies["Position"] = pd.Categorical(pitch["Position"], categories=pos)
df_dummies = pd.get_dummies(df_dummies, columns=["Position"], drop_first=True)

In [32]:
# print([f'{var}_sqrt' for var in asimetric_vars])

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

# Variables opcionales (skewness moderado)
# sqrt_vars = ['Interceptions', 'Yellow_cards', 'Transfer_value', 'Exp_contr', 'Minutes_pl', 'Minutes_pl_BC', 'NP_goals', 'Pen_goals', 'Pen_goals_BC', 'Assists', 'GCA_BC', 'Red_cards']
# for var in sqrt_vars: 
#     df_dummies[f'{var}_sqrt'] = df_dummies[var].apply(np.sqrt)

# Variables numéricas a estandarizar
numeric_vars = ['Transfer_value', 'Exp_contr', 'Age', 'Minutes_pl', 'Minutes_pl_BC', 'NP_goals', 'Pen_goals', 'Pen_goals_BC', 'Assists', 'Interceptions', 'GCA_BC', 'Yellow_cards', 'Red_cards']
df_dummies[numeric_vars] = StandardScaler().fit_transform(df_dummies[numeric_vars])

# ['Transfer_value', 'Exp_contr', 'Minutes_pl_BC', 'NP_goals', 'Pen_goals', 'Pen_goals_BC', 'Assists', 'Interceptions', 'GCA_BC', 'Yellow_cards', 'Red_cards']


In [33]:
from scipy.stats import skew

asimetric_vars = ['Transfer_value', 'Exp_contr', 'Minutes_pl_BC', 'NP_goals', 'Pen_goals', 'Pen_goals_BC', 'Assists', 'Interceptions', 'GCA_BC', 'Yellow_cards', 'Red_cards']

# Calcular asimetría para cada variable
skewness = df_dummies[[f'{var}_sqrt' for var in asimetric_vars]].apply(skew)
print(skewness)

# Filtrar solo variables con asimetría significativa (|skew| > 0.5)
vars_to_transform = skewness[abs(skewness) > 0.5].index.tolist()

Transfer_value_sqrt    1.438918
Exp_contr_sqrt         0.373368
Minutes_pl_BC_sqrt     0.202628
NP_goals_sqrt          0.517561
Pen_goals_sqrt         1.744483
Pen_goals_BC_sqrt      3.161119
Assists_sqrt           0.195474
Interceptions_sqrt     0.287321
GCA_BC_sqrt            0.961118
Yellow_cards_sqrt     -0.258038
Red_cards_sqrt         0.815147
dtype: float64


In [16]:
df_dummies.head()

,Transfer_value,Exp_contr,Age,Minutes_pl,Minutes_pl_BC,NP_goals,Pen_goals,Pen_goals_BC,Assists,Interceptions,...,Position_Left-Back,Position_Central Midfield,Position_Centre-Back,Position_Right Winger,Position_Attacking Midfield,Position_Defensive Midfield,Position_Left Winger,Position_Second Striker,Position_Left Midfield,Position_Right Midfield
0,-0.568888,-0.929287,-0.428140,-0.877141,-0.531844,-0.176562,-0.423149,-0.266365,-0.834010,-0.918112,...,0,0,0,0,0,0,0,0,0,0
1,0.362289,0.195872,-1.303969,0.148080,0.473316,-0.354793,-0.423149,-0.266365,-0.674372,-0.063875,...,1,0,0,0,0,0,0,0,0,0
2,-0.543624,-1.188939,-1.887855,-1.240273,-0.893641,-0.800370,-0.423149,-0.266365,-0.834010,-1.027630,...,1,0,0,0,0,0,0,0,0,0
3,-0.579716,-1.015838,0.447689,-1.113117,-0.893641,-0.354793,-0.011242,-0.266365,-0.834010,-0.918112,...,0,0,0,0,0,0,0,0,0,0
4,-0.341508,1.061379,1.323519,1.071257,1.964820,-0.265678,0.400665,1.003403,0.123820,0.637037,...,0,1,0,0,0,0,0,0,0,0


In [372]:
df_dummies.to_excel("reduced_pitch.xlsx", index=False)

In [106]:
# print(df_dummies[numeric_vars].mean())  # Debe ser cercano a 0
# print(df_dummies[numeric_vars].std())   # Debe ser cercano a 1

In [265]:
# df_dummies.to_excel("train_pitch.xlsx", index=False) 
['Transfer_value_sqrt', 'Age', 'Exp_contr_sqrt', 'Minutes_pl_BC_sqrt', 'NP_goals_sqrt', 'Pen_goals_sqrt', 'Pen_goals_BC_sqrt', 'Assists_sqrt', 'Interceptions_sqrt', 'GCA_BC_sqrt', 'Yellow_cards_sqrt', 'Red_cards_sqrt']

In [17]:
dic_vars = {
# 'vars1': ['Age', 'Exp_contr_sqrt', 'Minutes_pl_BC_sqrt', 'NP_goals_sqrt', 'Pen_goals_sqrt', 'Pen_goals_BC_sqrt', 'Assists_sqrt', 'Interceptions_sqrt', 'GCA_BC_sqrt', 'Yellow_cards_sqrt', 'Red_cards_sqrt', 'Position_Right Midfield', 'Position_Centre-Forward', 'Position_Left-Back',
#        'Position_Central Midfield', 'Position_Centre-Back', 'Position_Right Winger', 'Position_Attacking Midfield', 'Position_Defensive Midfield', 'Position_Left Winger', 'Position_Second Striker', 'Position_Left Midfield'],

'vars1': ['Age', 'Exp_contr', 'Minutes_pl_BC', 'NP_goals', 'Pen_goals', 'Pen_goals_BC', 'Assists', 'Interceptions', 'GCA_BC', 'Yellow_cards', 'Red_cards', 'Position_Right Midfield', 'Position_Centre-Forward', 'Position_Left-Back',
       'Position_Central Midfield', 'Position_Centre-Back', 'Position_Right Winger', 'Position_Attacking Midfield', 'Position_Defensive Midfield', 'Position_Left Winger', 'Position_Second Striker', 'Position_Left Midfield'],

    
'vars2': ['Reduced_team_from_Inter', 'Reduced_team_from_AC Milan', 'Reduced_team_from_Juventus', 'Reduced_team_from_Napoli', 'Reduced_team_from_Paris SG', 'Reduced_team_from_Real Madrid', 'Reduced_team_from_Barcelona', 'Reduced_team_from_Atlético Madrid',
         'Reduced_team_from_Bayern Munich', 'Reduced_team_from_Bor. Dortmund', 'Reduced_team_from_Chelsea', 'Reduced_team_from_Arsenal', 'Reduced_team_from_Man City', 'Reduced_team_from_Man Utd', 'Reduced_team_from_Liverpool', 'Reduced_country_from_Inglaterra', 
         'Reduced_country_from_Francia', 'Reduced_country_from_España', 'Reduced_country_from_Italia', 'Reduced_country_from_Alemania', 'Reduced_country_from_Portugal', 'Reduced_country_from_Países Bajos', 'Reduced_country_from_Argentina', 'Reduced_country_from_Brasil'],
}

In [13]:
# def corr(df, variables):
#     df_sub = df[variables]
#     plt.figure(figsize=(16, 12))
#     sns.heatmap(df_sub.corr(), xticklabels=variables, yticklabels=variables, annot=True, cmap='coolwarm', annot_kws={'size': 12}) #
#     plt.xticks(fontsize=14)  # Etiquetas del eje x
#     plt.yticks(fontsize=14)
    
# corr(df_dummies, vars1+vars2)

In [18]:
def multiple_lr(df, indep_vars):
    X = df[indep_vars]
    y = df["Transfer_value"]  # Transfer_value_sqrt

    # Añadir una constante al conjunto de entrenamiento para el término independiente (intercepto)
    X = sm.add_constant(X)

    # Entrenar el modelo de Ordinary Least Squares (OLS) de statsmodels
    model_sm = sm.OLS(y, X).fit()
    
    model_sm_robust = model_sm.get_robustcov_results(cov_type='HC3')

    # Obtener los p-valores de los coeficientes
    p_values = model_sm_robust.pvalues

    # Crear un DataFrame con los coeficientes y sus p-valores
    p_values_df = pd.DataFrame({
        "Variable": X.columns,
        "P-value": p_values
    })
    
    mse = model_sm_robust.mse_resid
    print(f"MSE: {mse}")

    # Ordenar el DataFrame por el valor absoluto del coeficiente
#     print(coefficients_df.sort_values(by="Coeficiente", key=abs, ascending=False))

    
    return (model_sm_robust, p_values_df)

In [19]:
# p_values

In [20]:
(summ, p_values) = multiple_lr(df_dummies, dic_vars['vars1'])
# summ.summary()

MSE: 0.5564708518894486


In [21]:
ns = 1
significant_variables = dic_vars['vars1']

while ns > 0:
    summ, p_values_df = multiple_lr(df_dummies, significant_variables)

    alpha = 0.05

    # Identificar las variables significativas (excluyendo la constante)
    significant_variables = p_values_df[(p_values_df["P-value"] <= alpha) & (p_values_df["Variable"] != "const")]["Variable"].tolist()
    ns = len(p_values_df) - len(significant_variables) - 1
#     print(ns)

significant_variables_next = significant_variables
print(summ.summary())

MSE: 0.5564708518894486
MSE: 0.556002060716808
MSE: 0.5563375394660334
                            OLS Regression Results                            
Dep. Variable:         Transfer_value   R-squared:                       0.445
Model:                            OLS   Adj. R-squared:                  0.444
Method:                 Least Squares   F-statistic:                     90.53
Date:                Sat, 10 May 2025   Prob (F-statistic):          2.66e-150
Time:                        14:13:25   Log-Likelihood:                -3542.2
No. Observations:                3151   AIC:                             7104.
Df Residuals:                    3141   BIC:                             7165.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------

In [33]:
# significant_variables + dic_vars['vars2']
# list(significant_variables + dic_vars['vars2'])

In [22]:
ns = 1
significant_variables = list(significant_variables_next + dic_vars['vars2'])

while ns > 0:
    summ, p_values_df = multiple_lr(df_dummies, significant_variables)

    alpha = 0.05

    # Identificar las variables significativas (excluyendo la constante)
    significant_variables = p_values_df[(p_values_df["P-value"] <= alpha) & (p_values_df["Variable"] != "const")]["Variable"].tolist()
    ns = len(p_values_df) - len(significant_variables) - 1
#     print(ns)

significant_variables_next = significant_variables
print(summ.summary())

MSE: 0.5094664149349719
MSE: 0.5106498302153316
MSE: 0.5137272013317965
                            OLS Regression Results                            
Dep. Variable:         Transfer_value   R-squared:                       0.490
Model:                            OLS   Adj. R-squared:                  0.486
Method:                 Least Squares   F-statistic:                     44.71
Date:                Sat, 10 May 2025   Prob (F-statistic):          8.99e-161
Time:                        14:13:29   Log-Likelihood:                -3410.7
No. Observations:                3151   AIC:                             6865.
Df Residuals:                    3129   BIC:                             6999.
Df Model:                          21                                         
Covariance Type:                  HC3                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------